In [12]:
# Import libraries and dataset

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns



# Adjust pandas display options
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent truncation


df = pd.read_csv('/Users/joezhou/Downloads/Mentum Assignment Data/Customer-churn-records.csv',sep=",")



In [6]:
# Preprocessing from assignment 1






,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned
0,1,15634602,Hargrave,619,NSW,Female,42,2,0.00,1,1,1,101348.88,1,1,2,DIAMOND,464
1,2,15647311,Hill,608,QLD,Female,41,1,83807.86,1,0,1,112542.58,0,1,3,DIAMOND,456
2,3,15619304,Onio,502,NSW,Female,42,8,159660.80,3,1,0,113931.57,1,1,3,DIAMOND,377
3,4,15701354,Boni,699,NSW,Female,39,1,0.00,2,0,0,93826.63,0,0,5,GOLD,350
4,5,15737888,Mitchell,850,QLD,Female,43,2,125510.82,1,1,1,79084.10,0,0,5,GOLD,425


In [11]:
# Part 1.1: Logistic regression model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder


# Preprocessing: Converting categorical variables to numerical using Label Encoding
le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])  # 0 for Female, 1 for Male
df['Geography'] = le.fit_transform(df['Geography'])  # Label encoding Geography
# df['Card Type'] = le.fit_transform(df['Card Type']) 


# Drop columns that are not needed (e.g., RowNumber, CustomerId, Surname)
df.drop(columns=['RowNumber', 'CustomerId', 'Surname','Card Type'], inplace=True)

# Features (X) and target (y)
X = df.drop('Exited', axis=1)  # Features
y = df['Exited']  # Target variable

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the feature variables
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Logistic Regression model
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train_scaled, y_train)

# Make predictions
y_pred = log_reg.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print the results
print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)




KeyError: "['RowNumber', 'CustomerId', 'Surname'] not found in axis"

In [ ]:
# Part 1.2: Random Forrest Model


# Importing necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder

# Load the dataset
df = pd.read_csv('your_dataset.csv')

# Preprocessing: Converting categorical variables to numerical using Label Encoding
le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])  # 0 for Female, 1 for Male
df['Geography'] = le.fit_transform(df['Geography'])  # Label encoding Geography

# Drop columns that are not needed (e.g., RowNumber, CustomerId, Surname)
df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], inplace=True)

# Features (X) and target (y)
X = df.drop('Exited', axis=1)  # Features
y = df['Exited']  # Target variable

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the feature variables
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Random Forest model
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train_scaled, y_train)

# Make predictions
y_pred = rf_classifier.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print the results
print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)



In [ ]:
# Part 1.3: Gradient boost machines

# Importing necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder

# Load the dataset
df = pd.read_csv('your_dataset.csv')

# Preprocessing: Converting categorical variables to numerical using Label Encoding
le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])  # 0 for Female, 1 for Male
df['Geography'] = le.fit_transform(df['Geography'])  # Label encoding Geography

# Drop columns that are not needed (e.g., RowNumber, CustomerId, Surname)
df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], inplace=True)

# Features (X) and target (y)
X = df.drop('Exited', axis=1)  # Features
y = df['Exited']  # Target variable

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the feature variables
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Gradient Boosting Classifier
gb_classifier = GradientBoostingClassifier(random_state=42)
gb_classifier.fit(X_train_scaled, y_train)

# Make predictions
y_pred = gb_classifier.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print the results
print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)



# Part 1:
# explore logistic regression and ensemble models like Random Forests and Gradient Boosting Machines (GBMs). 
# Use these models to predict customer churn and compare their performance to each other.
# Evaluate the performance of your models using ROC/AUC, precision, and recall, and visualise these metrics for easier interpretation. 
# Check for signs of over- or underfitting and data leakage.

# Part 2
# Now it’s time to improve your models. 
# Apply hyperparameter tuning to optimise the parameters of your models. 
# Experiment with feature engineering techniques to create new features from the existing data. 
# Consider if data quality enhancements or sampling techniques could help improve your model's performance.


# Part 3
# Focus on maintaining the quality of your model. 
# Outline an approach for monitoring your model's performance over time and detect any drift in the underlying data. Also, start thinking about how to communicate your findings.
# Create visualisations that clearly and effectively communicate your model's performance and the key factors contributing to customer churn. Save these to your Python notebook.
